## 1. Problem Statement
The objective of this notebook is to perform an initial exploratory data analysis (EDA)
and data cleaning process on a global species distribution dataset.
The goal is to identify data quality issues and prepare a clean dataset
for downstream geospatial and comparative analysis.

## 2. Data Loading
The dataset used in this analysis was obtained from the World Spider Catalog.
It contains taxonomic and distribution-related information for spider species
at a global level.

In [264]:
# library import
import pandas as pd

In [265]:
# uploading a CSV File
df = pd.read_csv('/Users/yayo/Documents/GitHub/jumping_spider_salticidae/data/raw/world_spider_catalog.csv')

## 3. Exploratory Data Analysis (EDA)

### 3.1 Structure and data types

In [266]:
# dataframe dimensions (rows and columns)
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

Number of rows: 64291
Number of columns: 13


In [267]:
# columns of the dataframe
df.columns

Index(['speciesId', 'species_legacy_id', 'species_lsid', 'family', 'genus',
       'species', 'subspecies', 'author', 'year', 'parentheses',
       'distribution', 'validSpeciesId', 'taxonStatus'],
      dtype='object')

In [268]:
# data loaded
# display the first few rows of the dataframe
df.head(3)

,speciesId,species_legacy_id,species_lsid,family,genus,species,subspecies,author,year,parentheses,distribution,validSpeciesId,taxonStatus
0,6625,1.0,urn:lsid:nmbe.ch:spidersp:000896,Actinopodidae,Actinopus,caraiba,NaN,Simon,1889,1,Venezuela,NaN,VALID
1,6626,2.0,urn:lsid:nmbe.ch:spidersp:000898,Actinopodidae,Actinopus,crassipes,NaN,Keyserling,1891,1,"Brazil, Paraguay, Argentina",NaN,VALID
2,6627,3.0,urn:lsid:nmbe.ch:spidersp:000899,Actinopodidae,Actinopus,cucutaensis,NaN,Mello-Leitão,1941,0,"Colombia, Venezuela, Brazil",NaN,VALID


In [269]:
# identify the data type of each variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64291 entries, 0 to 64290
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   speciesId          64291 non-null  int64  
 1   species_legacy_id  64139 non-null  float64
 2   species_lsid       64291 non-null  object 
 3   family             64291 non-null  object 
 4   genus              64291 non-null  object 
 5   species            64291 non-null  object 
 6   subspecies         937 non-null    object 
 7   author             64291 non-null  object 
 8   year               64291 non-null  object 
 9   parentheses        64291 non-null  int64  
 10  distribution       64142 non-null  object 
 11  validSpeciesId     10600 non-null  float64
 12  taxonStatus        64291 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.4+ MB


### 3.2 Missing Data Analysis

In [270]:
df.isna().sum().sort_values(ascending=False)

subspecies           63354
validSpeciesId       53691
species_legacy_id      152
distribution           149
speciesId                0
species_lsid             0
family                   0
genus                    0
species                  0
author                   0
year                     0
parentheses              0
taxonStatus              0
dtype: int64

In [271]:
(df.isna().mean() * 100).sort_values(ascending=False)

subspecies           98.542564
validSpeciesId       83.512467
species_legacy_id     0.236425
distribution          0.231759
speciesId             0.000000
species_lsid          0.000000
family                0.000000
genus                 0.000000
species               0.000000
author                0.000000
year                  0.000000
parentheses           0.000000
taxonStatus           0.000000
dtype: float64

In [272]:
# Count missing values per variable
missing_values = df.isnull().sum()

# Calculate completeness percentage
completeness = pd.DataFrame(
    100 - (missing_values / len(df) * 100),
    columns=["completeness"]
)

# Reset index and rename columns
completeness = (
    completeness
    .reset_index()
    .rename(columns={"index": "variable"})
)

# Sort variables by completeness (ascending)
completeness = completeness.sort_values(
    by="completeness",
    ascending=True
)

completeness

,variable,completeness
6,subspecies,1.457436
11,validSpeciesId,16.487533
1,species_legacy_id,99.763575
10,distribution,99.768241
0,speciesId,100.000000
2,species_lsid,100.000000
3,family,100.000000
4,genus,100.000000
5,species,100.000000
7,author,100.000000


### 3.3 Duplicate Inspection

In [273]:
df.duplicated().sum()

np.int64(3)

In [274]:
df[df.duplicated(keep=False)]

,speciesId,species_legacy_id,species_lsid,family,genus,species,subspecies,author,year,parentheses,distribution,validSpeciesId,taxonStatus
64144,73199,NaN,urn:lsid:nmbe.ch:spidersp:066575,Theraphosidae,Euathlus,kupal,NaN,Allegue & Ferretti,2025,0,Argentina,NaN,VALID
64145,73199,NaN,urn:lsid:nmbe.ch:spidersp:066575,Theraphosidae,Euathlus,kupal,NaN,Allegue & Ferretti,2025,0,Argentina,NaN,VALID
64239,73300,NaN,urn:lsid:nmbe.ch:spidersp:066669,Agelenidae,Afrotrix,reginaldi,NaN,"Haddad, Zamani & Marusik",2026,0,South Africa,NaN,VALID
64240,73300,NaN,urn:lsid:nmbe.ch:spidersp:066669,Agelenidae,Afrotrix,reginaldi,NaN,"Haddad, Zamani & Marusik",2026,0,South Africa,NaN,VALID
64255,73323,NaN,urn:lsid:nmbe.ch:spidersp:066688,Clubionidae,Femorbiona,pinglong,NaN,Wu & Xu,2026,0,China,NaN,VALID
64256,73323,NaN,urn:lsid:nmbe.ch:spidersp:066688,Clubionidae,Femorbiona,pinglong,NaN,Wu & Xu,2026,0,China,NaN,VALID


### 3.4 Data Completeness Assessment

In [275]:
# se verifica la completitud de las variables (valores nulos existentes)
# se utiliza la función .sum() para sumar los elementos que están vacíos (.isnull())
nulos = df.isnull().sum()

# calculo del porcentaje de completitud
completitud = pd.DataFrame(100 - (nulos / len(df) * 100))

# reset_index se utiliza para resetear los índices
# inplace = True sirve para evitar la repetición
completitud.reset_index(inplace = True)
# se coloan encebezados
completitud = completitud.rename(columns = {"index":"variable",0:"completitud"})

# ordenamos las columnas con menor completitud en adelante
completitud.sort_values(by ='completitud', ascending = True)
completitud

,variable,completitud
0,speciesId,100.000000
1,species_legacy_id,99.763575
2,species_lsid,100.000000
3,family,100.000000
4,genus,100.000000
5,species,100.000000
6,subspecies,1.457436
7,author,100.000000
8,year,100.000000
9,parentheses,100.000000


## 4. Data Cleaning

### 4.1 Duplicate Removal

In [276]:
df = df.drop_duplicates(keep='first')

In [277]:
df.duplicated().sum()

np.int64(0)

### 4.2 Column Removal

In [278]:
df = df.drop(columns=['speciesId', 'species_legacy_id', 
                      'species_lsid','subspecies', 'parentheses', 
                      'validSpeciesId', 'taxonStatus'])

### 4.3 Creatio of Taxonomic Binomial Name

In [279]:
df['genus']=df['genus'].str.strip().str.title()
df['species']=df['species'].str.strip().str.lower() 
df['binomial_name']=df['genus']+' '+df['species']

### 4.4 Data Type Validation

In [280]:
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['family'] = df['family'].astype('category')
df['genus'] = df['genus'].astype('category')

### 4.5 Post-cleaning Validation

In [281]:
# dimensión del dataframe con eliminacion de datos duplicados
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

Número de filas: 64288
Número de columnas: 7


In [282]:
df.columns

Index(['family', 'genus', 'species', 'author', 'year', 'distribution',
       'binomial_name'],
      dtype='object')

In [283]:
print("Duplicados restantes:", df.duplicated().sum())
print("Duplicados en binomial_name:", df['binomial_name'].duplicated().sum())
print("Total valores nulos restantes:", df.isnull().sum().sum())

Duplicados restantes: 167
Duplicados en binomial_name: 1201
Total valores nulos restantes: 150


In [284]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64288 entries, 0 to 64290
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   family         64288 non-null  category
 1   genus          64288 non-null  category
 2   species        64288 non-null  object  
 3   author         64288 non-null  object  
 4   year           64287 non-null  float64 
 5   distribution   64139 non-null  object  
 6   binomial_name  64288 non-null  object  
dtypes: category(2), float64(1), object(4)
memory usage: 3.4+ MB


## 5. Export

In [285]:
# Guardar el dataframe en un archivo CSV
df.to_csv('/Users/yayo/Documents/GitHub/jumping_spider_salticidae/data/processed/wsc_cleaned.csv', index=False)

print("DataFrame guardado en wsc_cleaned.csv")

DataFrame guardado en wsc_cleaned.csv
